<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #4: Pre-Procesamiento de textos y Featuring Engineering 
`Fecha de entrega: Marzo 18, 2021. (Antes del inicio de la próxima clase).`

In [3]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install --user -U nltk

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [2]:
import re
import pandas as pd
from collections import Counter
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords_sp = stopwords.words('spanish')

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\XboxL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\XboxL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Punto 1: Pre-Procesamiento

- `[5 pts]` Leer el archivo `dialogos.csv` usando `pandas`
- `[10 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números y sin palabras vacias
- `[7 pts]` Usando la nueva columna, ¿cuáles son las 10 palabras más usadas? 

In [3]:
def listToString(lista):
    texto = ""   
    for palabra in lista: 
        #if(palabra.pos_ != 'NOUN'):
        texto += ' '+palabra.text   
    return texto.strip()

In [4]:
dialogos = pd.read_csv("Recursos/dialogos.csv",encoding='utf-8')
dialogos["pre-procesado"] = ""

In [5]:
from nltk.stem.snowball import SnowballStemmer
spanishStemmer = SnowballStemmer("spanish")
import es_core_news_sm

nlp = es_core_news_sm.load()
patron = r'[(!-@). , ¿? (0-9)¡]'

for i in range(dialogos["Locución"].size):
    dialogo = dialogos["Locución"][i]
    #Texto en minuscula
    dialogo = dialogo.lower()
    #Limpieza de caracteres y numeros
    dialogo = re.sub(patron , " " ,dialogo)
    #Obtencion de la palabra
    dialogo = spanishStemmer.stem(dialogo)
    #Union del Array Generado
    dialogo = listToString(nlp(dialogo))
    dialogos["pre-procesado"][i] = dialogo

dialogos

,Locución,pre-procesado
0,Francamente no,francamente n
1,"Lo del canadiense. Por favor, como que vosotro...",lo del canadiense por favor como que vosot...
2,Tienes razón. A lo mejor así te liberas de tu ...,tienes razon a lo mejor asi te liberas de tu...
3,"Pues, tú sabrás",pues tu sabras
4,Para mí que fue Krieger,para mi que fue krieg
...,...,...
954,"Yo qué sé, digo yo qué será",yo que se digo yo que s
955,Lo han entendido mal. Las joyas que venden en ...,lo han entendido mal las joyas que venden en...
956,Oooo... Me llamo Vivian,oooo me llamo vivi
957,¿De dónde? ¿Del coño de tu madre?,de donde del coño de tu madre


In [6]:
#Eliminacion palabras vacías
dialogos["pre-procesado"] = dialogos["pre-procesado"].apply(
                                                            lambda x : " ".join(
                                                                                [ w 
                                                                                    for w in word_tokenize(x)
                                                                                        if not w in stopwords_sp]
                                                                                )
                                                            )
dialogos

,Locución,pre-procesado
0,Francamente no,francamente n
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tirariais alli si pud
2,Tienes razón. A lo mejor así te liberas de tu ...,razon mejor asi liberas energia nag
3,"Pues, tú sabrás",pues sabras
4,Para mí que fue Krieger,krieg
...,...,...
954,"Yo qué sé, digo yo qué será",digo s
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...
956,Oooo... Me llamo Vivian,oooo llamo vivi
957,¿De dónde? ¿Del coño de tu madre?,coño madre


,Locución,pre-procesado
0,Francamente no,francamente
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa
3,"Pues, tú sabrás",pues sabrás
4,Para mí que fue Krieger,krieger
...,...,...
954,"Yo qué sé, digo yo qué será",sé digo
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...
956,Oooo... Me llamo Vivian,oooo llamo vivian
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre


In [7]:
palabras= Counter(dialogos["pre-procesado"].str.cat(sep=" ").split())
i=1
for val in palabras.most_common(10):
    print(f"{val[0]}:{val[1]}")
    i+=1

si:130
bien:32
asi:30
mas:29
oh:28
vamos:26
sarcasmobol:24
aqui:24
solo:22
bueno:22


[('bien', 36),
 ('si', 34),
 ('cómo', 34),
 ('así', 30),
 ('oh', 28),
 ('vamos', 26),
 ('sarcasmobol', 24),
 ('aquí', 24),
 ('bueno', 24),
 ('lana', 22)]

# Punto 2: Representación vectorial

- `[15 pts]` Crear una bolsa de palabras (BoW) del corpus usando la columna pre-procesada (Usando la función de `sklearn`)
- `[10 pts]` ¿Cuántas palabras hay en el vocabulario? (Usando la función de `sklearn`)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(binary=True)
bow_rep = count_vect.fit_transform(dialogos["pre-procesado"])
dialogos['bow'] = [row for row in bow_rep.toarray()]
dialogos

,Locución,pre-procesado,bow
0,Francamente no,francamente n,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tirariais alli si pud,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Tienes razón. A lo mejor así te liberas de tu ...,razon mejor asi liberas energia nag,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"Pues, tú sabrás",pues sabras,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Para mí que fue Krieger,krieg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
954,"Yo qué sé, digo yo qué será",digo s,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
956,Oooo... Me llamo Vivian,oooo llamo vivi,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
957,¿De dónde? ¿Del coño de tu madre?,coño madre,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


,Locución,pre-procesado,bow
0,Francamente no,francamente,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"Pues, tú sabrás",pues sabrás,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,Para mí que fue Krieger,krieger,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...
954,"Yo qué sé, digo yo qué será",sé digo,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
956,Oooo... Me llamo Vivian,oooo llamo vivian,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [9]:
len(count_vect.vocabulary_)

2244

2115

# Punto 3: 🤔

- `[3pts]` ¿Cuándo es una mejor idea usar una bolsa de n-gramas en vez de una bolsa de palabras?

La bolsa de n-gramas es mejor utilizarla cuando debemos tener en consideración el orden en el que se encuentran las palabras en el texto que vamos a analizar.

# BONUS Punto 4: 😜

- `[2pt]` ¿A qué pertenecen los dialogos de ese archivo? 

South Park Sarcasmobol